In [1]:
#! ./venv/bin/python3

import yaml
import os.path
import os
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from collections import Counter
import re
from pathlib import Path
from openai import OpenAI

from emotional_playlist_generator import EmotionalPlaylistGenerator

if not os.path.isfile('./secrets.yaml'):
    print('No secrets file - aborting')
    exit(1)

with open("./secrets.yaml", "r") as f:
    try:
        data = yaml.safe_load(f)
        spotify_auth = data['spotify']
        openai_key = data['openai']['secret']
    except yaml.YAMLError as exc:
        print(exc)
        exit(1)

generator = EmotionalPlaylistGenerator(api_key=openai_key)

print('Authenticating, please wait...')
auth_manager = SpotifyOAuth(client_id=spotify_auth['client_id'], client_secret=spotify_auth['secret'], redirect_uri='http://localhost:5000', scope='user-top-read')
sp = spotipy.Spotify(auth_manager=auth_manager)
print('Auth successful!')

artists = sp.current_user_top_artists(limit=50, time_range='medium_term')

genre_count = Counter()
genre_to_top_artists = {}

for artist in artists['items']:
    genre_count.update(artist['genres'])
    for genre in artist['genres']:
        if genre in genre_to_top_artists:
            genre_to_top_artists[genre].append((artist['id'], artist['name']))
        else:
            genre_to_top_artists[genre] = [(artist['id'], artist['name'])]

top_five_artists = []
for genre, _ in genre_count.most_common(15):
    if len(top_five_artists) >= 5:
        break
    for artist in genre_to_top_artists[genre]:
        if artist not in top_five_artists:
            top_five_artists.append(artist)
            break

#Comment out whichever 
        
#INPUT TEXT
# #-----------------------------------------------------------------------------------
# journal_entry = input("Enter your jounal entry: ")
# #-----------------------------------------------------------------------------------


#INPUT AUDIO
#-----------------------------------------------------------------------------------
speech_file_path = Path.cwd() / "speech.mp3"
# Initialise the OpenAI client
client = OpenAI()

text_in = input("Enter your jounal entry (for audio conversion): ")
# Create the TTS request
response = client.audio.speech.create(
    model="tts-1",
    voice="alloy",
    input=text_in #Test text for text to speech
)

# Save the response audio to a file
response.stream_to_file(speech_file_path)

generator = EmotionalPlaylistGenerator(api_key=openai_key)
journal_entry = generator.transcribe_audio('speech.mp3')
os.remove(speech_file_path) #Delete audio file after use
#-----------------------------------------------------------------------------------

emotion_summary = generator.analyse_emotion(journal_entry)
print(emotion_summary)
music_parameters = generator.get_music_parameters(emotion_summary)
print(music_parameters)

# Extract Spotify parameters using regular expressions
spotify_params = {}
for line in music_parameters.split('\n'):
    match = re.match(r'^.*target_(\w+).*=(.*)$', line)
    if match:
        param, value = match.groups()
        spotify_params[f'target_{param}'] = float(value)

# Display summary of emotions and top genres and artists
print('Stats:')
print(f'To summarise: {emotion_summary}')
for param, value in spotify_params.items():
    print(f'You\'re feeling {value * 100}% {param.replace("target_", "")}.')
print(f'Your top 10 genres are: {", ".join([g for g, _ in genre_count.most_common(10)])}')
print(f'Your top 5 artists, spread across your top 5ish genres, are: {", ".join([a for _, a in top_five_artists])}')

limit = input('How many tracks to recommend? (at most, 1-50): ')
limit = int(''.join(c for c in limit if c.isdigit()))

# Get recommendations from Spotify
reccs = sp.recommendations(seed_artists=[id for id, _ in top_five_artists], limit=limit, **spotify_params)

print('We recommend:')
for track in reccs['tracks']:
    print(f'  - "{track["name"]}" by {[a["name"] for a in track["artists"]]}')


Authenticating, please wait...
Auth successful!


/tmp/ipykernel_229418/2215460836.py:80: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


The user is feeling extremely excited and has had a fantastic day.
target_valence = 0.85
target_danceability = 0.9
target_energy = 0.8
target_acousticness = 0.2
target_instrumentalness = 0.1
target_liveness = 0.7
target_speechiness = 0.5
target_tempo = 120.00
Stats:
To summarise: The user is feeling extremely excited and has had a fantastic day.
You're feeling 85.0% valence.
You're feeling 90.0% danceability.
You're feeling 80.0% energy.
You're feeling 20.0% acousticness.
You're feeling 10.0% instrumentalness.
You're feeling 70.0% liveness.
You're feeling 50.0% speechiness.
You're feeling 12000.0% tempo.
Your top 10 genres are: rap, rock, permanent wave, modern rock, alternative rock, melodic rap, pop, uk hip hop, orchestral soundtrack, soundtrack
Your top 5 artists, spread across your top 5ish genres, are: Travis Scott, Radiohead, Arctic Monkeys, Royal Blood, Muse
We recommend:
  - "Here Comes Your Man" by ['Pixies']
  - "SLUT ME OUT" by ['NLE Choppa']
  - "Broke In A Minute" by ['Tor